# sPyTial for Z3: Making Constraint Solving Visual

**Why sPyTial revolutionizes Z3 constraint solving**

Traditional Z3 work is purely textual - you write constraints, get satisfiable/unsatisfiable results, and examine models through print statements. sPyTial transforms this by making the structure of your constraints, models, and solving process spatially visible.

## The Problem with Traditional Z3 Workflows

**Traditional approach:**
```python
solver = Solver()
solver.add(constraints...)
if solver.check() == sat:
    print(solver.model())  # Just text output
```

**Problems:**
- ❌ No visual understanding of constraint relationships  
- ❌ Models are just text dumps
- ❌ Hard to debug complex constraint interactions
- ❌ Difficult to understand solver state
- ❌ No insight into constraint structure

**sPyTial approach:**
- ✅ Visual constraint architecture
- ✅ Spatial model representation
- ✅ Clear solver state visualization
- ✅ Structural debugging capabilities

In [ ]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from z3 import *
from spytial import diagram, orientation, group, atomColor, attribute
import json

## Demo 1: Simple Constraint Problem

Let's start with a basic problem and compare traditional vs sPyTial approaches.

In [ ]:
# Traditional Z3 approach
print("=== TRADITIONAL Z3 APPROACH ===")
x, y = Ints('x y')
solver = Solver()
solver.add(x + y == 10, x > 0, y > 0, x < 8)

print(f"Constraints: {solver.assertions()}")
result = solver.check()
print(f"Result: {result}")

if result == sat:
    model = solver.model()
    print(f"Model: {model}")
    print(f"x = {model[x]}, y = {model[y]}")
else:
    print("No solution found")
    
print("\n👎 Traditional approach only gives you text!")

In [ ]:
# sPyTial approach - visualize the model structure
print("=== SPYTIAL APPROACH ===")

if result == sat:
    model = solver.model()
    print("Z3 Model Structure:")
    diagram(model, method="inline")
    
    # Also visualize the constraint solving process
    constraint_info = {
        'variables': {'x': str(x), 'y': str(y)},
        'constraints': [str(c) for c in solver.assertions()],
        'solution': {str(var): str(model[var]) for var in model if str(var) in ['x', 'y']},
        'solver_state': str(result)
    }
    
    print("\nConstraint Solving Process:")
    diagram(constraint_info, method="inline")
    
print("\n👍 sPyTial shows you the STRUCTURE of your solution!")

## Demo 2: Complex Scheduling Problem

Let's tackle a real-world constraint problem: task scheduling with dependencies.

In [ ]:
# Complex scheduling problem
print("=== TASK SCHEDULING PROBLEM ===")

# Define tasks with durations
tasks = ['setup', 'coding', 'testing', 'review', 'deploy']
durations = {'setup': 2, 'coding': 5, 'testing': 3, 'review': 2, 'deploy': 1}

# Create start time variables for each task
start_times = {task: Int(f'start_{task}') for task in tasks}
end_times = {task: start_times[task] + durations[task] for task in tasks}

# Create solver and add constraints
scheduler = Solver()

# All tasks must start at non-negative times
for task in tasks:
    scheduler.add(start_times[task] >= 0)

# Dependencies: setup -> coding -> testing -> review -> deploy
scheduler.add(start_times['coding'] >= end_times['setup'])
scheduler.add(start_times['testing'] >= end_times['coding'])
scheduler.add(start_times['review'] >= end_times['testing'])
scheduler.add(start_times['deploy'] >= end_times['review'])

# Project deadline: must finish within 15 time units
scheduler.add(end_times['deploy'] <= 15)

print(f"Number of constraints: {len(scheduler.assertions())}")
print(f"Variables: {list(start_times.keys())}")

In [ ]:
# Traditional approach - just text output
print("TRADITIONAL: Text-only solution")
print("=" * 40)

if scheduler.check() == sat:
    schedule_model = scheduler.model()
    print("Schedule found:")
    for task in tasks:
        start = schedule_model[start_times[task]].as_long()
        end = start + durations[task]
        print(f"{task}: start={start}, end={end}, duration={durations[task]}")
else:
    print("No valid schedule found")
    
print("\n👎 Hard to visualize the schedule structure from text!")

In [ ]:
# sPyTial approach - visual schedule structure
print("SPYTIAL: Visual schedule structure")
print("=" * 40)

if scheduler.check() == sat:
    schedule_model = scheduler.model()
    
    # Create a structured representation of the schedule
    schedule_structure = {
        'project_timeline': {},
        'task_details': {},
        'dependencies': {},
        'constraints': {
            'deadline': 15,
            'total_constraints': len(scheduler.assertions()),
            'solver_status': 'satisfiable'
        }
    }
    
    # Fill in the timeline
    for task in tasks:
        start = schedule_model[start_times[task]].as_long()
        end = start + durations[task]
        
        schedule_structure['project_timeline'][task] = {
            'start': start,
            'end': end,
            'duration': durations[task]
        }
        
        schedule_structure['task_details'][f'{task}_execution'] = {
            'time_slot': f'{start}-{end}',
            'resource_needed': durations[task]
        }
    
    # Add dependencies  
    deps = [('setup', 'coding'), ('coding', 'testing'), ('testing', 'review'), ('review', 'deploy')]
    for i, (pred, succ) in enumerate(deps):
        schedule_structure['dependencies'][f'dependency_{i+1}'] = {
            'predecessor': pred,
            'successor': succ,
            'constraint': f'{succ} starts after {pred} ends'
        }
    
    # Visualize the complete schedule structure
    diagram(schedule_structure, method="inline")
    
    print("\n👍 sPyTial reveals the schedule architecture!")
    print("- Timeline structure is spatially clear")
    print("- Dependencies are visually obvious")
    print("- Constraint relationships are apparent")
else:
    print("No valid schedule found")

## Demo 3: Debugging Unsatisfiable Constraints

One of sPyTial's biggest advantages is helping debug why constraints can't be satisfied.

In [ ]:
# Create an unsatisfiable problem
print("=== DEBUGGING UNSATISFIABLE CONSTRAINTS ===")

a, b, c = Ints('a b c')
debug_solver = Solver()

# Add conflicting constraints
debug_solver.add(a + b == 10)  # Constraint 1
debug_solver.add(a > 5)        # Constraint 2
debug_solver.add(b > 5)        # Constraint 3 - conflicts with 1&2
debug_solver.add(a + b + c == 20)  # Constraint 4
debug_solver.add(c < 5)        # Constraint 5 - might conflict with 4

result = debug_solver.check()
print(f"Solver result: {result}")

In [ ]:
# Traditional debugging - very limited
print("TRADITIONAL: Limited debugging info")
print("=" * 40)

if result == unsat:
    print("Constraints are unsatisfiable")
    print("Assertions:")
    for i, assertion in enumerate(debug_solver.assertions()):
        print(f"  {i+1}: {assertion}")
    print("\n👎 Traditional approach: you have to manually figure out conflicts!")
else:
    print("Problem is satisfiable")

In [ ]:
# sPyTial debugging - visualize constraint structure
print("SPYTIAL: Visual constraint analysis")
print("=" * 40)

# Create a structured view of the constraint problem
constraint_analysis = {
    'variables': {
        'a': {'type': 'Int', 'appears_in': ['eq1', 'ineq1', 'eq2']},
        'b': {'type': 'Int', 'appears_in': ['eq1', 'ineq2', 'eq2']},
        'c': {'type': 'Int', 'appears_in': ['eq2', 'ineq3']}
    },
    'constraints': {
        'eq1': {'formula': 'a + b == 10', 'type': 'equality', 'status': 'active'},
        'ineq1': {'formula': 'a > 5', 'type': 'inequality', 'status': 'active'},
        'ineq2': {'formula': 'b > 5', 'type': 'inequality', 'status': 'conflicts_with_eq1'},
        'eq2': {'formula': 'a + b + c == 20', 'type': 'equality', 'status': 'depends_on_eq1'},
        'ineq3': {'formula': 'c < 5', 'type': 'inequality', 'status': 'might_conflict'}
    },
    'conflict_analysis': {
        'primary_conflict': {
            'involved_constraints': ['eq1', 'ineq1', 'ineq2'],
            'explanation': 'If a > 5 and b > 5, then a + b > 10, contradicting a + b == 10',
            'severity': 'fatal'
        },
        'secondary_issues': {
            'constraint_chain': ['eq1', 'eq2', 'ineq3'],
            'potential_problem': 'c value might be over-constrained'
        }
    },
    'solver_state': {
        'result': str(result),
        'total_constraints': len(debug_solver.assertions()),
        'satisfiable': False
    }
}

# Visualize the constraint analysis
diagram(constraint_analysis, method="inline")

print("\n👍 sPyTial advantages for debugging:")
print("- Constraint relationships are visually clear")
print("- Conflict sources are spatially highlighted")
print("- Variable usage patterns are obvious")
print("- Debugging workflow is structured and visual")

## Demo 4: Multi-Solver Comparison

sPyTial excels at comparing different solving approaches or solver configurations.

In [ ]:
# Compare different solving strategies
print("=== MULTI-SOLVER COMPARISON ===")

# Same problem, different solvers/tactics
x, y, z = Ints('x y z')

# Solver 1: Default
solver1 = Solver()
solver1.add(x + y + z == 15, x > 0, y > 0, z > 0, x * y < 20, z < 8)

# Solver 2: With different tactics
solver2 = SolverFor('LIA')  # Linear Integer Arithmetic
solver2.add(x + y + z == 15, x > 0, y > 0, z > 0, x * y < 20, z < 8)

# Solve with both
result1 = solver1.check()
result2 = solver2.check()

print(f"Default solver: {result1}")
print(f"LIA solver: {result2}")

In [ ]:
# Traditional comparison - just separate outputs
print("TRADITIONAL: Separate solver outputs")
print("=" * 40)

if result1 == sat:
    model1 = solver1.model()
    print(f"Solver 1 model: {model1}")
    
if result2 == sat:
    model2 = solver2.model()
    print(f"Solver 2 model: {model2}")
    
print("\n👎 Hard to compare solver approaches side-by-side!")

In [ ]:
# sPyTial comparison - unified view
print("SPYTIAL: Unified solver comparison")
print("=" * 40)

# Create comparative analysis structure
solver_comparison = {
    'problem_definition': {
        'variables': ['x', 'y', 'z'],
        'constraints': [
            'x + y + z == 15',
            'x > 0, y > 0, z > 0', 
            'x * y < 20',
            'z < 8'
        ],
        'complexity': 'mixed linear/nonlinear'
    },
    'solver_approaches': {
        'default_solver': {
            'type': 'General SMT',
            'result': str(result1),
            'model': {str(var): str(solver1.model()[var]) for var in [x, y, z]} if result1 == sat else None,
            'strategy': 'general_purpose'
        },
        'lia_solver': {
            'type': 'Linear Integer Arithmetic',
            'result': str(result2), 
            'model': {str(var): str(solver2.model()[var]) for var in [x, y, z]} if result2 == sat else None,
            'strategy': 'specialized_linear'
        }
    },
    'analysis': {
        'both_satisfiable': result1 == sat and result2 == sat,
        'models_match': None,  # Would need to compare if both sat
        'performance_notes': 'LIA solver optimized for linear constraints'
    }
}

# Check if models match (if both satisfiable)
if result1 == sat and result2 == sat:
    model1, model2 = solver1.model(), solver2.model()
    models_identical = all(
        model1[var].as_long() == model2[var].as_long() 
        for var in [x, y, z]
    )
    solver_comparison['analysis']['models_match'] = models_identical

# Visualize the comparison
diagram(solver_comparison, method="inline")

print("\n👍 sPyTial reveals solver comparison insights:")
print("- Side-by-side solver analysis")
print("- Clear problem structure vs solver approaches")
print("- Unified view of different solving strategies")
print("- Structural comparison of results")

## Summary: sPyTial's Advantages for Z3

| Traditional Z3 | sPyTial + Z3 |
|---------------|-------------|
| Text-only models | Spatial model structure |
| Manual constraint debugging | Visual constraint analysis |
| Isolated solver runs | Comparative solver views |
| No structural insight | Clear architectural patterns |
| Hard to understand relationships | Spatial constraint relationships |

### When sPyTial Transforms Z3 Work:

✅ **Learning constraint solving** - See structure, not just text  
✅ **Debugging unsatisfiable problems** - Visual conflict analysis  
✅ **Complex constraint systems** - Understand architectural patterns  
✅ **Solver comparison** - Side-by-side analysis  
✅ **Teaching/explaining solutions** - Spatial intuition  
✅ **Documentation** - Visual constraint specifications  
✅ **Code reviews** - Structural constraint understanding  

### The Key Insight

**Traditional Z3**: You write constraints, get answers  
**sPyTial + Z3**: You *see* constraint architecture and solution structure

sPyTial doesn't replace Z3's solving power - it makes Z3's results comprehensible and debuggable through spatial visualization. This is especially powerful for:

- **Educational use**: Students can see constraint relationships
- **Industrial applications**: Debug complex constraint systems
- **Research**: Understand solver behavior patterns
- **Collaboration**: Share visual constraint architectures

# But is this useful? Not really

You can register a DataInstanceBuilder for Z3 that provides some directions on HOW things should be translated to boxes and arrows.



In [4]:
from spytial.provider_system import DataInstanceProvider, data_provider
from z3 import ModelRef

@data_provider(priority=10)
class ArithmeticTheoryProvider(DataInstanceProvider):
    """Custom provider for Z3 models in the arithmetic theory."""

    def can_handle(self, obj):
        return isinstance(obj, ModelRef)  # Check if the object is a Z3 model

    def provide_atoms_and_relations(self, obj, walker_func):
        atoms = []
        relations = []

        # Add variables as atoms
        for decl in obj.decls():
            try:
                var_name = decl.name()
                var_expr = decl()  # Convert the declaration into a Z3 expression
                var_value = obj.eval(var_expr, model_completion=True)  # Evaluate the expression
                atom_id = walker_func._get_id(var_name)
                atom = {
                    "id": atom_id,
                    "type": "ArithmeticVariable",
                    "label": f"{var_name} = {var_value}"
                }
                atoms.append(atom)
            except Exception as e:
                print(f"Skipping declaration {decl}: {e}")

        # Add constraints as relations
        # Example: Add relations for constraints (extend this for real constraints)
        for decl in obj.decls():
            if decl.name() == "x":  # Example: Add a relation for x
                x_id = walker_func._get_id("x")
                y_id = walker_func._get_id("y")
                relations.append(("constraint", x_id, y_id))

        return atoms, relations

# Use the custom ArithmeticTheoryProvider to visualize the Z3 model
# The diagram function will now use the registered provider for better DEFAULT visualization

diagram(z3_model)


Skipping declaration y: b'ast is not an expression'
Skipping declaration x: b'ast is not an expression'
Error during data instance building: list indices must be integers or slices, not str
Object: [y = 1, x = 9]
